In [1]:
import os
import sys
import torch
import time

base_directory = "../"
sys.path.insert(0, base_directory)

output_directory = "./output/noise-tests/sd2-notebook"
os.makedirs(output_directory, exist_ok=True)

from stable_diffusion2.latent_diffusion import LatentDiffusion
from stable_diffusion2.stable_diffusion import StableDiffusion
from stable_diffusion2.utils.model import *
from stable_diffusion2.utils.utils import SectionManager as section
from stable_diffusion2.utils.utils import *
from stable_diffusion2.model.clip.clip_embedder import CLIPTextEmbedder



from stable_diffusion2.model.unet.unet import UNetModel

from pathlib import Path
from os.path import join

device = get_device()

INFO: Using CUDA device: NVIDIA GeForce RTX 3080 Ti


In [2]:
latent_diffusion_model = LatentDiffusion(linear_start=0.00085,
            linear_end=0.0120,
            n_steps=1000,
            latent_scaling_factor=0.18215,
            device = device)


INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.


In [3]:
latent_diffusion_model.load_submodel_tree()

In [4]:
get_memory_status()

Total: 12287 MiB
Free: 6932 MiB
Used: 5355 MiB


(tensor(6932), tensor(12287))

In [5]:
def logistic_distribution(loc, scale):
    base_distribution = torch.distributions.Uniform(0, 1)
    transforms = [torch.distributions.transforms.SigmoidTransform().inv, torch.distributions.transforms.AffineTransform(loc=loc, scale=scale)]
    logistic = torch.distributions.TransformedDistribution(base_distribution, transforms)
    return logistic
noise_fn = lambda shape, device = device: logistic_distribution(loc=0.0, scale=0.5).sample(shape).to(device)

In [6]:
get_memory_status()

Total: 12287 MiB
Free: 6932 MiB
Used: 5355 MiB


(tensor(6932), tensor(12287))

In [7]:
ddim_eta = 0.00000
stable_diffusion_model = StableDiffusion(device = device, ddim_eta=ddim_eta)
stable_diffusion_model.initialize_from_model(latent_diffusion_model)

INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.


In [8]:
get_memory_status()

Total: 12287 MiB
Free: 6478 MiB
Used: 5809 MiB


(tensor(6478), tensor(12287))

In [9]:
temperature = 1.0
imgs = stable_diffusion_model.generate_images(
    prompt = 'A woman with flowers in her hair in a courtyard, in the style of Frank Frazetta',
    seed = 2982,
    noise_fn = noise_fn,
    temperature = temperature,
    )

save_images(imgs, join(output_directory, f'test_temp{temperature}_eta{ddim_eta:.3f}.png'))

In [10]:
get_memory_status()

Total: 12287 MiB
Free: 2758 MiB
Used: 9529 MiB


(tensor(2758), tensor(12287))

In [11]:
del imgs
torch.cuda.empty_cache()

In [12]:
get_memory_status()

Total: 12287 MiB
Free: 5844 MiB
Used: 6443 MiB


(tensor(5844), tensor(12287))

In [40]:
ddim_eta = 0.1
stable_diffusion_model.ddim_eta = ddim_eta

In [41]:
get_memory_status()

Free: 5714 MiB
Total: 12287 MiB
Used: 6573 MiB


(tensor(5714), tensor(12287))

In [42]:
temperature = 1.0
imgs = stable_diffusion_model.generate_images(
    prompt = 'A woman with flowers in her hair in a courtyard, in the style of Frank Frazetta',
    seed = 2982,
    noise_fn = noise_fn,
    temperature = temperature,
    )

save_images(imgs, join(output_directory, f'test_temp{temperature}_eta{ddim_eta:.3f}.png'))

In [44]:
print(get_memory_status())
del imgs
torch.cuda.empty_cache()
print(get_memory_status())

Free: 5714 MiB
Total: 12287 MiB
Used: 6573 MiB
(tensor(5714), tensor(12287))


NameError: name 'imgs' is not defined

In [46]:
temperature = 1.0
images = []
for i in range(4):
    ddim_eta = 0.1 + i * 0.1
    stable_diffusion_model.ddim_eta = ddim_eta
    imgs = stable_diffusion_model.generate_images(
        prompt = 'A woman with flowers in her hair in a courtyard, in the style of Frank Frazetta',
        seed = 2982,
        noise_fn = noise_fn,
        temperature = temperature,
        )
    save_images(imgs, join(output_directory, f'test_temp{temperature}_eta{ddim_eta:.3f}.png'))
    images.append(imgs)

In [ ]:
print(get_memory_status())
del imgs
torch.cuda.empty_cache()
print(get_memory_status())

In [51]:
print(get_memory_status())
del images
torch.cuda.empty_cache()
print(get_memory_status())

Free: 4626 MiB
Total: 12287 MiB
Used: 7661 MiB
(tensor(4626), tensor(12287))


NameError: name 'images' is not defined

In [15]:
temperature = 1.0
images = []
for i in range(4):
    temperature += 0.1
    imgs = stable_diffusion_model.generate_images(
        prompt = 'A woman with flowers in her hair in a courtyard, in the style of Frank Frazetta',
        seed = 2982,
        noise_fn = noise_fn,
        temperature = temperature,
        )
    save_images(imgs, join(output_directory, f'test_temp{temperature}_eta{ddim_eta:.3f}.png'))
    images.append(imgs)